In [1]:
import os
import sys

from neo4j import GraphDatabase
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.config import RAW_DIR, PROCESSED_DIR

uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

weekly_stats = pd.read_parquet(PROCESSED_DIR / "player_stats_2018_2024.parquet")
injuries = pd.read_parquet(RAW_DIR / "injuries_2018_2024.parquet")
profiles = pd.read_parquet(PROCESSED_DIR / "player_profiles.parquet")

/home/nolan/anaconda3/envs/nfl_llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.rag_eval_functions import generate_injury_queries, generate_profile_queries, generate_weekly_stats_queries

weekly_queries = generate_weekly_stats_queries(weekly_stats, n_samples=100)
profile_queries = generate_profile_queries(profiles, n_samples=100)
injury_queries = generate_injury_queries(injuries, n_samples=100)
all_queries = weekly_queries + profile_queries + injury_queries

In [3]:
from src.rag_functions import hybrid_retrieve_and_answer
from src.rag_eval_functions import evaluate_label_answer, evaluate_numeric_answer, evaluate_text_answer

results = []

for q in all_queries:
    if q["index"] == "weekly-stats":
        df_lookup = weekly_stats
    elif q["index"] == "profiles":
        df_lookup = profiles
    else:
        df_lookup = injuries

    # 2. Get ground truth using the lambda
    truth = q["truth_lookup"](df_lookup)

    # 3. Run your RAG pipeline
    print(q["query"])
    prediction = await hybrid_retrieve_and_answer(q["query"])

    # 4. Evaluate prediction
    scores = q["eval_fn"](prediction, truth)

    # 5. Save everything
    results.append({
        "index": q["index"],
        "query": q["query"],
        "ground_truth": truth,
        "prediction": prediction,
        **scores
    })


How many passes did Kenyan Drake attempt in Week 2, 2018?
How many passing yards did Chris Godwin have in Week 16, 2018?
How many times was Gary Brightwell sacked in Week 12, 2022?
How many carries did Josh Jacobs have in Week 4, 2020?
How many passes did Courtland Sutton attempt in Week 12, 2021?
How many carries did Noah Fant have in Week 13, 2019?
How many times was DJ Moore sacked in Week 1, 2023?
How many interceptions did Laviska Shenault throw in Week 18, 2021?
How many catches did Noah Gray have in Week 1, 2024?
How many catches did Jameson Williams have in Week 13, 2023?
How many receiving touchdowns did Wendell Smallwood score in Week 5, 2019?
How many times was Garrett Gilbert sacked in Week 5, 2019?
How many catches did Nate Adkins have in Week 18, 2024?
How many passing touchdowns did Alec Pierce throw in Week 18, 2024?
How many receiving yards did Rondale Moore have in Week 14, 2021?
How many catches did Tua Tagovailoa have in Week 14, 2020?
How many fantasy points did An

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: DRAFTED_BY)} {position: line: 1, column: 66, offset: 65} for query: "MATCH (p:Player {first_name: 'Jimmy', last_name: 'Morrissey'})-[:DRAFTED_BY]->(t:Team) RETURN t.name AS team_drafted"


How tall is Jimmy Gaines?
What position does Doug Jones play?
How tall is Brandon Hoyte?
What year did Seth Olsen enter the NFL?
What number does Maxie Baughan wear?
What year did Tucker Addington enter the NFL?
What year was Cap Boso drafted?
What round was Jamain Stephens drafted?
How tall is Dwight Clark?
What year did Nikko Remigio enter the NFL?
What year was Clay Brown drafted?
Which team does LaQuan Williams currently play for?
Which team drafted David Pittman?
Which team does Brandyn Dombrowski currently play for?
What year did Alex Highsmith enter the NFL?
Which team does Brandon London currently play for?
What is the weight of Damiere Byrd?
What is the weight of Michael Palardy?
What position does Mikel Jones play?
What position does Dan Mozes play?
Which team does Phil Micech currently play for?
What is the weight of Antonio Smith?
What year did Joel Hilgenberg enter the NFL?
What position does Pete Najarian play?
What year was Rick Engles drafted?
What is the weight of Brad

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: DRAFTED_BY)} {position: line: 1, column: 61, offset: 60} for query: "MATCH (p:Player {first_name: 'Doug', last_name: 'Evans'})-[:DRAFTED_BY]->(t:Team) RETURN t.name AS team_name"


What position does Isaiah Johnson play?
What is the weight of Steve Collier?
Which team does Corey Alston currently play for?
What year did Joe Felton enter the NFL?
What year was Vince Marrow drafted?
How tall is Doug Riesenberg?
What is the weight of Mao Tosi?
Which team drafted Jack Squirek?
What pick was Len Bell in the draft?
What year did Jon Embree enter the NFL?
What is the weight of Marcus Buggs?
What number does Tyrell Peters wear?
How tall is Nate Noel?
What is the weight of Ray Wagner?
What year did Deneric Prince enter the NFL?
How tall is Marty Hochertz?
How tall is Carl Hilton?
What is the weight of Santonio Holmes?
What number does David Nelson wear?
How tall is Phillip Supernaw?
What position does Ray Butler play?
What round was Montori Hughes drafted?
Which team does Errict Rhett currently play for?
Which team drafted Derrel Gofourth?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: DRAFTED_BY)} {position: line: 1, column: 46, offset: 45} for query: "MATCH (p:Player {name: 'Derrel Gofourth'})-[:DRAFTED_BY]->(t:Team) RETURN t.name AS team_name"


What year did Leonard Wheeler enter the NFL?
Which team does Herkie Walls currently play for?
What year was Corn Elder drafted?
What was N'Keal Harry's injury status in Week 15, 2022?
Cypher parse error: Invalid \escape: line 2 column 39 (char 40)
What was Marcus Peters's injury status in Week 1, 2022?
What was Jordan Elliott's injury status in Week 6, 2024?
What was Rashaan Evans's injury status in Week 20, 2019?
What was Darrell Taylor's injury status in Week 10, 2024?
What was Zack Moss's injury status in Week 17, 2023?
What was David Morgan's injury status in Week 11, 2018?
What was Darren Sproles's injury status in Week 4, 2018?
What was Anfernee Jennings's injury status in Week 17, 2024?
What was DeAndre Hopkins's injury status in Week 17, 2022?
What was Jordan Poyer's injury status in Week 17, 2022?
What was Dwayne Harris's injury status in Week 7, 2019?
What was Jason Pierre-Paul's injury status in Week 7, 2021?
What was Rashard Robinson's injury status in Week 17, 2020?
What w

In [8]:
df_results = pd.DataFrame(results)
df_results.to_csv(RAW_DIR / "rag_kg_eval_results_full.csv", index=False)
df_results.head()

,index,query,ground_truth,prediction,exact,within_tol,fuzzy
0,weekly-stats,How many passes did Kenyan Drake attempt in We...,0,{'question': 'How many passes did Kenyan Drake...,1,1.0,NaN
1,weekly-stats,How many passing yards did Chris Godwin have i...,0.0,{'question': 'How many passing yards did Chris...,1,1.0,NaN
2,weekly-stats,How many times was Gary Brightwell sacked in W...,0.0,{'question': 'How many times was Gary Brightwe...,1,1.0,NaN
3,weekly-stats,How many carries did Josh Jacobs have in Week ...,15,{'question': 'How many carries did Josh Jacobs...,1,1.0,NaN
4,weekly-stats,How many passes did Courtland Sutton attempt i...,0,{'question': 'How many passes did Courtland Su...,1,1.0,NaN


In [9]:
df_results.tail()

,index,query,ground_truth,prediction,exact,within_tol,fuzzy
236,injuries,What was Marlon Davidson's injury status in We...,Out,{'question': 'What was Marlon Davidson's injur...,1,NaN,NaN
237,injuries,What was Dennis Kelly's injury status in Week ...,Questionable,{'question': 'What was Dennis Kelly's injury s...,1,NaN,NaN
238,injuries,What was Marquise Goodwin's injury status in W...,Out,{'question': 'What was Marquise Goodwin's inju...,1,NaN,NaN
239,injuries,What was Landon Collins's injury status in Wee...,Out,{'question': 'What was Landon Collins's injury...,1,NaN,NaN
240,injuries,What was Craig Reynolds's injury status in Wee...,Questionable,{'question': 'What was Craig Reynolds's injury...,1,NaN,NaN


In [11]:
summary = df_results.groupby("index")[["exact", "within_tol", "fuzzy"]].mean()
summary

,exact,within_tol,fuzzy
index,,,
injuries,0.97561,NaN,NaN
profiles,0.00000,NaN,0.75
weekly-stats,0.89000,0.95,NaN
